In [1]:
from dysh.spectra.spectrum import Spectrum
import numpy as np
from scipy.stats import norm
import astropy.units as u
import matplotlib.pyplot as plt
from dysh.fits.sdfitsload import SDFITSLoad
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.spectra.scan import PSScan, GBTPSScan
import pandas
from copy import deepcopy
pandas.set_option('display.max_rows', 100)
from astropy.modeling.fitting import LevMarLSQFitter,LinearLSQFitter
from specutils import Spectrum1D, SpectrumList,SpectralRegion
from astropy.nddata import StdDevUncertainty
from specutils.fitting import fit_continuum
import matplotlib.pyplot as plt
from dysh.spectra import dcmeantsys
from dysh.plot.specplot import SpectrumPlot
from astropy.io import fits
from astropy.table import Table
import pandas as pd
#%matplotlib notebook

-------------

# 1. GBTFITSLoad and position-switch calibration testing
change to use your specific `datadir`

In [2]:
datadir = '/data/gbt'
#f = datadir + '/examples/misc/ngc5291.fits'
f = datadir + '/examples/onoff-L/data/TGBT21A_501_11.raw.vegas.fits'
#f = datadir + 'examples/nod-KFPA/data/TGBT22A_503_02.raw.vegas/TGBT22A_503_02.raw.vegas.A.fits'
#f = datadir + '/examples/mixed-fs-ps/data/AGBT16B_225_05/AGBT16B_225_05.raw.vegas/AGBT16B_225_05.raw.vegas.A.fits'
p = GBTFITSLoad(f)
p.info()

Filename: /data/gbt/examples/onoff-L/data/TGBT21A_501_11.raw.vegas.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      12   ()      
  1  SINGLE DISH    1 BinTableHDU    245   6040R x 74C   ['32A', '1D', '22A', '1D', '1D', '1D', '32768E', '16A', '6A', '8A', '1D', '1D', '1D', '4A', '1D', '4A', '1D', '1I', '32A', '32A', '1J', '32A', '16A', '1E', '8A', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '1D', '8A', '1D', '1D', '12A', '1I', '1I', '1D', '1D', '1I', '1A', '1I', '1I', '16A', '16A', '1J', '1J', '22A', '1D', '1D', '1I', '1A', '1D', '1E', '1D', '1D', '1D', '1D', '1D', '1A', '1A', '8A', '1E', '1E', '16A', '1I', '1I', '1I']   


In [3]:
p.summary()

,SCAN,OBJECT,VELOCITY,PROC,PROCSEQN,RESTFREQ,DOPFREQ,# IF,# POL,# INT,# FEED,AZIMUTH,ELEVATIO
0,152.0,NGC2415,3784.0,OnOff,1.0,1.617185,1.420406,5,2,151,1,286.218008,41.62843
1,153.0,NGC2415,3784.0,OnOff,2.0,1.617185,1.420406,5,2,151,1,286.886521,41.118134


-----------------------------
# debugging total power stuff

In [4]:
scanlist = [6]
df = p._ptable[0]
tpscan = p.rawspectra(0)[list(df[(df["SCAN"].isin(scanlist)) & (df['CAL']=='F') & (df['SIG']=='T')].index)]
np.shape(tpscan)

(0, 32768)

In [5]:
if False:
    class TPScan(object):
        """
        Holds a total power scan, honestly how is this different than a PS scan

        Parameters
        ----------
            sdfits : ~SDFITSLoad
                input SDFITSLoad object (or derivative)
            scan: int
                scan number
            sig_state : str
                one of 'SIG' or 'REF' to indicate if this is the signal or reference scan
            scanrows : list-like
                the list of rows in `sdfits` corresponding to sig_state integrations 
            bintable : int
                the index for BINTABLE in `sdfits` containing the scans
        """
        def __init__(self, sdfits, scan, sig_state, scanrows, bintable):
            self._sdfits = sdfits # parent class
            self._status = 0 #@TODO make these an enumeration, possibly dict
            #                           # ex1:
            self._nint = 0              # 11
            self._npol = 0              #  2
            self._sig = None            # 44
            self._ref= None             # 44
            self._calibrated = None     # 22
            self._timeaveraged = None   #  2
            self._polaveraged = None    #  1
            self._bintable_index = bintable
            self._nrows = len(scanrows['SIG'])
            print(f"TPSCAN nrows = {self.nrows}")

        @property
        def status(self):
            """Status flag, will be used later for undo"""
            return self._status

        @property
        def nrows(self):
            """The number of rows in this Scan"""
            return self._nrows

        @property
        def npol(self):
            """The number of polarizations in this Scan"""
            return self._npol

        #@TODO write calibrated data to a FITS? file.
        #def write(self,filename,format,**kwargs):

        def __len__(self):
            return self._nrows


In [7]:
d=p.calonoff_rows(scans=152,bintable=0,ifnum=0,plnum=0)
len(d['ON'])

151

In [11]:
tps = p.gettp(scan=152,sig=True,cal=False,bintable=0,ifnum=0,plnum=0)
tps.tsys

151
TPSCAN nrows = 151


array([17.45805392, 17.4667357 , 17.51221928, 17.4754147 , 17.27344124,
       17.23664637, 17.42674345, 17.50969773, 17.36458603, 17.41340577,
       17.27404075, 17.36085714, 17.28454048, 17.30618109, 17.14631619,
       16.99965576, 17.54035873, 17.21348618, 17.39550931, 17.38196485,
       17.4854426 , 17.42790778, 17.15065431, 17.20435199, 17.44797606,
       17.15114418, 17.30894688, 17.3141454 , 17.10243963, 17.4810351 ,
       17.29237157, 17.23753315, 17.5246924 , 17.3108259 , 17.19219528,
       17.09618503, 17.34704484, 17.13676429, 17.38384526, 17.48417142,
       17.22028478, 17.50097571, 17.32987002, 17.26765015, 17.40924251,
       17.19881764, 17.4766609 , 17.21482952, 17.36519109, 17.46644011,
       17.25167018, 17.18425595, 17.250671  , 17.17091688, 17.13623695,
       17.36259599, 17.24760407, 17.27883821, 17.25393638, 17.19083945,
       17.36394905, 17.14476191, 17.48024963, 17.17207704, 17.36090432,
       17.42563464, 17.5077979 , 17.37680797, 17.28915614, 17.18

-----------------------------

In [ ]:

p.summary(verbose=False)

In [ ]:
p.summary(verbose=True)

In [ ]:
help(p.summary)

## Try writing scans or records to FITS files and reading them back in


In [ ]:
scanlist = [152]

In [ ]:
p.write_scans("/tmp/scans.fits",overwrite=True,checksum=True,output_verify='fix',scans=scanlist)

In [ ]:
p.write("/tmp/foobar.fits",overwrite=True,rows=[11,15,37,99],bintable=0)
p.write("/tmp/onerows.fits",overwrite=True,rows=15,bintable=0)

In [ ]:
x1 = fits.open("/tmp/scans.fits")
x1.info()

In [ ]:
x1[1].header

In [ ]:
x1 = GBTFITSLoad("/tmp/scans.fits")
#x1._ptable[0]
x1._hdu.info()

In [ ]:
x1._hdu[1].header

In [ ]:
x1 = GBTFITSLoad("/tmp/foobar.fits")
#x1.nrows(0),x1._hdu[1].header
x1.summary()

In [ ]:
x1 = GBTFITSLoad("/tmp/onerows.fits")
x1.nrows(0),x1._hdu[1].header
x1.summary()

In [ ]:
spec4 = p.getspec(4)
sp = SpectrumPlot(spec4)
sp.plot()

## Get a position switch scan and calibrate it.
The default ifnum=0, plnum=0

In [ ]:
# eventually this will have calibrate=True as the default, so you don't need to call calibrate().
psscan = p.getps(152)

In [ ]:
psscan.calibrate()

### the raw calibrated numpy array

In [ ]:
psscan._calibrated[0]

### A single integration of the calibrated Spectrum

In [ ]:
spec1 = psscan.calibrated(0)
spec1.unit == u.K

In [ ]:
sp = SpectrumPlot(spec1)
sp.plot(xaxis_unit="km/s")

In [ ]:
if False:
    fig,ax = plt.subplots()
    ax.set_ylim([-4,4])
    ax.plot(spec1.spectral_axis,spec1.flux)
    ax.set_xlabel("Frequency (GHz)")
    ax.set_ylabel(r"$T_a$ (K)")

In [ ]:
## ignore this
if False:
    flx = np.mean(psscan._calibrated[np.arange(0,len(psscan._calibrated),4)])
    fig,ax = plt.subplots()
    ax.set_ylim([-4,4])
    ax.plot(psscan.calibrated(0).spectral_axis,flx)
    all(psscan.calibrated(0).spectral_axis==psscan.calibrated(3).spectral_axis),all(psscan.calibrated(4).spectral_axis==psscan.calibrated(7).spectral_axis)

### The average spectrum.  
Note this will be encapsulated in a average() method.
Compare this with GBTIDL `getps, 152`

In [ ]:
avg = Spectrum(spectral_axis=psscan.calibrated(0).spectral_axis,flux=np.mean(psscan._calibrated)*u.K)
avg.plot()
print(f"T_sys = {np.mean(psscan.tsys)}")

### Try a different polarization and IF and use the OFF scan number
GBTIDL `getps, 153, ifnum=1, plnum=1`

In [ ]:
psscan = p.getps(152,plnum=None,ifnum=1)
psscan.calibrate()

In [ ]:
avg = Spectrum(spectral_axis=psscan.calibrated(0).spectral_axis,flux=np.mean(psscan._calibrated)*u.K)
avg.plot()
print(f"T_sys = {np.mean(psscan.tsys)}")

--------------------------------

# NOD KPFA Data  -- THIS DOESN'T WORK

In [ ]:
datadir = "/data/gbt/"
f = datadir + 'examples/nod-KFPA/data/TGBT22A_503_02.raw.vegas/TGBT22A_503_02.raw.vegas.A.fits'
pnod = GBTFITSLoad(f)
pnod.info()

In [ ]:
p._hdu[1].header

In [ ]:
"INSTRUMEN" in p._ptable[0].columns

In [ ]:
pnod.getspec(10,0).meta

In [ ]:
pnod.summary(verbose=True)[0:10]

In [ ]:
pnod._ptable[0].columns

In [ ]:
set(pnod._ptable[0]["INTNUM"]),set(pnod._ptable[0]["FDNUM"]),set(pnod._ptable[0]["FEED"]),set(pnod._ptable[0]["IFNUM"]),set(pnod._ptable[0]["SRFEED"]),set(pnod._ptable[0]["SAMPLER"])

-----
# Debugging from here down.

In [ ]:
df = p._ptable[0]
len(p.select('OBJECT','NGC2415',p._ptable[0]))//p.npol(0)

In [ ]:
psscan.meta

In [ ]:
p.index(0).iloc[psscan._scanrows["OFF"]]["CAL"]

In [ ]:
psscan.calibrate()

In [ ]:
df = p._ptable[0]
scans = [152,153]
df2=df[df["SCAN"].isin([152])]
df3=df[df["SCAN"].isin([153])]
df4=df[df["SCAN"].isin([156])]
un = df2.index.union(df3.index)
un2=df2.index
un3=df3.index
df["SCAN"].iloc[un2]
df3.reindex(un2).index

In [ ]:
on = [152,154,156]
off = [153,155,157]
scans = [152,155]
onrequested = set(on).intersection(scans)
offrequested = set(off).intersection(scans)
sons = list(onrequested.copy())
soffs = list(offrequested.copy())
missingoff = []
missingon = []
setoff = set(off)
seton = set(on)
for i in onrequested:
    expectedoff = i+1
    if len(setoff.intersection([expectedoff])) == 0:
        missingoff.append(expectedoff)
    else:
        soffs.append(expectedoff)
for i in offrequested:
    expectedon = i-1
    if len(seton.intersection([expectedon])) == 0:
        missingon.append(expectedon)
    else:
        sons.append(expectedon)
if len(missingoff) > 0:
    raise ValueError(f"For the requested ON scans {onrequested}, the OFF scans {missingoff} were not present in bintable {bintable}")
if len(missingon) > 0:
    raise ValueError(f"For the requested OFF scans {offrequested}, the ON scans {missingon} were not present in bintable {bintable}")
print("ON",sorted(sons))
print("OFF",sorted(soffs))

In [ ]:
scanlist=p.onoff_scan_list()
print(scanlist)
#scanlist = p.onoff_scan_list([152])
#print(scanlist)
scanlist = p.onoff_scan_list([153])
print(scanlist)
scanlist = p.onoff_scan_list([152,153])
print(scanlist)
scanlist = p.onoff_scan_list(152)
print(scanlist)

In [ ]:
print(sonoff(p.index(0)["SCAN"],p.index(0)["PROCSEQN"]))
myscans = set(sorted([153,155]))
scanlist=p.onoff_scan_list()
print(scanlist)
allscans = set(sorted(scanlist["ON"]+scanlist["OFF"]+[1,2,3,4,5]))
print(allscans)
z=allscans.intersection(myscans)
zz =allscans.difference(myscans)
zzz=myscans.difference(allscans)
print(z,zz,zzz)

In [ ]:
psscan = p.getps(scans=[152])

In [ ]:
df = p._ptable[0]
rows=list(df[df["SCAN"].isin(scans["ON"])].index)
df

In [ ]:
rows=p.onoff_rows()
rows

In [ ]:
show = ["SCAN", "PROCSEQN", "PROCSIZE", "_OBSTYPE"]
df=df[df.columns & show]
df[df["_OBSTYPE"]=="PSWITCHOFF"]

In [ ]:
dict(p._ptable[0].iloc[0])

In [ ]:
df["RESTFREQ"]/=1E9
df["RESTFREQ"]

In [ ]:
show = ["SCAN", "OBJECT", "OBSMODE", "VELOCITY", "PROCSCAN", "PROCSEQN", "PROCTYPE", "PROCSIZE", "RESTFREQ", "IFNUM","FEED", "AZIMUTH", "ELEVATIO", "FDNUM"] 
scan = [60,61]
_df = df[df.columns & show].reindex(columns=show)

In [ ]:
_df["OBSMODE"].str.split(':',expand=True)

In [ ]:

_df[(_df["SCAN"]>=scan[0]) & ( _df["SCAN"] <= scan[1])]

In [ ]:
df["SCAN"]

In [ ]:
df['PROCTYPE']